# 1. Data fetching

## Setup

In [1]:
import logging
import os
import sys

# set up module paths for imports
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)
src_path = os.path.abspath(os.path.join('..', 'src'))
sys.path.append(src_path)

# start logging system and set logging level
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info("Starting logger")

INFO:root:Starting logger


In [4]:
DATA_DIR = os.path.join('..', 'data')

## Getting the protocols URLs

In [5]:
PROTOCOL_URLS_FILE = os.path.join(DATA_DIR, 'protocol_urls.txt')

def get_protocols_urls(file_name):
    with open(file_name, 'r') as f:
        urls = [url.rstrip() for url in f]
    return urls


In [6]:
urls = get_protocols_urls(PROTOCOL_URLS_FILE)
len(urls)

100

In [7]:
urls[0]

'https://bio-protocol.org/e16'

## Fetching the protocols

In [19]:
import requests
import time

from tqdm import tqdm

BASE_URL = "https://bio-protocol.org/"

class BioProtocolScrapper():
    def __init__(self, output_dir, throttle_time=.5,
                 username=None, password=None):
        self.output_dir = output_dir
        self.throttle_time = throttle_time
        if username and password:
            self.login(username, password)
    
    def fetch_urls(self, url_list):
        pbar = tqdm(url_list)
        for url in pbar:
            pbar.set_description("Processing %s" % url)
            self.fetch_url(url)
            time.sleep(self.throttle_time)
    
    def fetch_url(self, url):
        filename = url.split('/')[-1] if '/' in url else url
        filename += ".html"
        response = requests.get(url)
        with open(os.path.join(self.output_dir, filename), 'w', encoding='utf-8') as f:
            f.write(response.text)
    
    def _login(self, user, password):
        payload = {'txtEmail': user, 'txtPassword': password}
        url = f'{BASE_URL}/ifrlogin.aspx/?sign=in&p=4'
        requests.post(url, data=payload)


In [20]:
OUTPUT_DIR = os.path.join(DATA_DIR, 'protocols')

scrapper = BioProtocolScrapper(OUTPUT_DIR)
scrapper.fetch_urls(urls)

Processing https://bio-protocol.org/e3436: 100%|██████████| 100/100 [03:42<00:00,  2.22s/it]
